# 📊 Step 1: Data Collection (CPU Only - FREE)

**NO GPU NEEDED** - Run this with CPU runtime to save money!

This notebook:
1. Clones C repositories from GitHub
2. Extracts functions and compiles to WAT
3. Saves dataset to Google Drive

**Time:** ~2-3 hours
**Cost:** FREE (CPU only)

After this completes, run **Step2_Train_Model.ipynb** with H100 GPU.

In [ ]:
#@title Configuration { display-mode: "form" }

TARGET_PAIRS = 10000  #@param {type:"integer"}
ENABLE_AUGMENTATION = True  #@param {type:"boolean"}

print(f"Target: {TARGET_PAIRS} pairs")
print(f"Augmentation: {'ON' if ENABLE_AUGMENTATION else 'OFF'}")
print("\n⚠️ Make sure you're using CPU runtime (Runtime → Change runtime type → None)")

In [ ]:
# Ready to collect data
print("✓ Starting data collection pipeline...")

In [ ]:
# Mount Google Drive first
from google.colab import drive
drive.mount('/content/drive')

DRIVE_PATH = '/content/drive/MyDrive/zerolang_data'
!mkdir -p {DRIVE_PATH}
print(f"\n✓ Data will be saved to: {DRIVE_PATH}")

In [ ]:
%%bash
# Install dependencies
echo "[1/3] Installing LLVM toolchain..."
apt-get update -qq
apt-get install -qq -y llvm lld clang > /dev/null 2>&1

echo "[2/3] Installing wasm-tools..."
cd /tmp
curl -sLO https://github.com/bytecodealliance/wasm-tools/releases/download/v1.244.0/wasm-tools-1.244.0-x86_64-linux.tar.gz
tar -xzf wasm-tools-1.244.0-x86_64-linux.tar.gz
cp wasm-tools-1.244.0-x86_64-linux/wasm-tools /usr/local/bin/
chmod +x /usr/local/bin/wasm-tools

echo "[3/3] Verifying installation..."
clang --version | head -1
wasm-tools --version

# Quick test
echo 'int add(int a, int b) { return a + b; }' > /tmp/test.c
clang --target=wasm32 -nostdlib -fuse-ld=lld -Wl,--no-entry -Wl,--export-all -o /tmp/test.wasm /tmp/test.c 2>/dev/null
if wasm-tools print /tmp/test.wasm > /dev/null 2>&1; then
    echo "✓ WASM compilation works!"
else
    echo "✗ WASM compilation failed"
    exit 1
fi

In [ ]:
# Upload project files from local
# Download project as ZIP: your local project-zero folder → compress to ZIP

from google.colab import files
import zipfile
import os
import shutil

print("📦 Upload project-zero.zip")
print("(Compress your local project-zero folder to ZIP and upload)")
print()

uploaded = files.upload()
zip_file = list(uploaded.keys())[0]

# Clean up and extract
!rm -rf /content/zerolang
with zipfile.ZipFile(zip_file, 'r') as z:
    z.extractall('/content/')
    
# Find and rename extracted folder
extracted = [d for d in os.listdir('/content') if 'project-zero' in d.lower() or 'zerolang' in d.lower()]
if extracted:
    src = f'/content/{extracted[0]}'
    if src != '/content/zerolang':
        shutil.move(src, '/content/zerolang')
        
%cd /content/zerolang

# Update clang path for Colab
!sed -i 's|/opt/homebrew/opt/llvm/bin/clang|clang|g' pipeline/collect_large.py 2>/dev/null || true
!sed -i 's|/opt/homebrew/opt/llvm/bin/clang|clang|g' pipeline/generator.py 2>/dev/null || true

print("✓ Project loaded successfully")

In [ ]:
%%time
# Run large-scale data collection
import os
os.makedirs('data', exist_ok=True)

aug_flag = "" if ENABLE_AUGMENTATION else "--no-augment"

print(f"Starting data collection...")
print(f"Target: {TARGET_PAIRS} pairs")
print(f"This will take 2-3 hours...\n")
print("="*60)

!python pipeline/collect_large.py \
    --target {TARGET_PAIRS} \
    --output data/large_dataset.jsonl \
    {aug_flag} \
    2>&1 | tee data/collection.log

In [ ]:
# Check results
import json
from pathlib import Path

data_file = Path('data/large_dataset.jsonl')
if data_file.exists():
    with open(data_file) as f:
        pairs = [json.loads(l) for l in f if l.strip()]
    
    print(f"="*60)
    print(f"Data Collection Results")
    print(f"="*60)
    print(f"Total pairs: {len(pairs)}")
    
    if pairs:
        avg_instr = sum(len(p['instruction']) for p in pairs) / len(pairs)
        avg_wat = sum(len(p['output']) for p in pairs) / len(pairs)
        augmented = sum(1 for p in pairs if p.get('metadata', {}).get('augmented', False))
        
        print(f"Original pairs: {len(pairs) - augmented}")
        print(f"Augmented pairs: {augmented}")
        print(f"Avg instruction: {avg_instr:.0f} chars")
        print(f"Avg WAT: {avg_wat:.0f} chars")
else:
    print("ERROR: No data file found!")

In [ ]:
# Split into train/val
!python pipeline/postprocess.py split data/large_dataset.jsonl \
    --train data/train_large.jsonl \
    --val data/val_large.jsonl \
    --val-ratio 0.1

# Convert to ChatML format
!python training/prepare_data.py data/train_large.jsonl -o data/train_chatml_large.jsonl -f chatml
!python training/prepare_data.py data/val_large.jsonl -o data/val_chatml_large.jsonl -f chatml

print("\nFinal files:")
!wc -l data/*_large*.jsonl

In [ ]:
# Save to Google Drive
import shutil

files_to_save = [
    'data/large_dataset.jsonl',
    'data/train_large.jsonl',
    'data/val_large.jsonl',
    'data/train_chatml_large.jsonl',
    'data/val_chatml_large.jsonl',
]

print(f"Saving to Google Drive: {DRIVE_PATH}")
for f in files_to_save:
    if Path(f).exists():
        shutil.copy(f, DRIVE_PATH)
        print(f"  ✓ {f}")

print(f"\n" + "="*60)
print(f"✅ Data saved to Google Drive!")
print(f"="*60)
print(f"\nNext step: Run Step2_Train_Model.ipynb with H100 GPU")

In [ ]:
# Summary
!ls -lh {DRIVE_PATH}/